In [2]:
import sys, os
sys.path.insert(0, os.path.abspath('../rse_watch'))
sys.path.insert(0, os.path.abspath('../'))
sys.path.insert(0, os.path.abspath('../../../data/model/'))
import pdf_parser, conf
from pdf_parser import *
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [13]:
dpef_path

WindowsPath('../../../data/model/DPEFs/Debug/edf_debug.pdf')

In [3]:
# loading
config = conf.DebugConfig('../../../data/model/')
companies_metadata_dict = get_companies_metadata_dict(config)
all_input_files = get_list_of_pdfs_filenames(config.dpef_dir)
all_input_files = [input_file for input_file in all_input_files if
                   input_file.name.split("_")[0] in companies_metadata_dict.keys()]
companies_metadata_dict = get_companies_metadata_dict(config)
dpef_path = all_input_files[0]
project_denomination = dpef_path.name.split("_")[0]
rse_ranges = companies_metadata_dict[project_denomination]["rse_ranges"]
rse_range = eval(rse_ranges)
rse_range

(0, 5)

In [45]:
%time
from collections import Counter

device, idx_first_page = get_raw_content_from_pdf(dpef_path, 
                                                  rse_range=rse_range)

# GROUPING BY COLUMN
column_text = OrderedDict()  # keep order of identification in the document.

APPROXIMATION_FACTOR = 10 # to allow for slight shifts at beg of aligned text
N_MOST_COMMON = 4 # e.g. nb max of columns of text that can be considered
LEFT_SECURITY_SHIFT = 20 # to include way more shifted text of previous column
counter = Counter()
item_holder = []

page_id = 0
item_index = 0
while "There are unchecked items in device.rows":
    
    # add the item to the list of the page
    (page_id, x_min, _, _, _, _) = device.rows[item_index]
    item_holder.append(device.rows[item_index])
    
    # count the x_min to find the x_groups
#     print(x_min, x_min//APPROXIMATION_FACTOR)
    counter[(x_min//APPROXIMATION_FACTOR)*APPROXIMATION_FACTOR]+=1
    
    # go to next item
    it_was_last_item = item_index == (len(device.rows)-1)
    if not it_was_last_item:
        item_index+=1
        (next_page_id, _, _, _, _, _) = device.rows[item_index]
        changing_page = (next_page_id > page_id)
        
    
    if changing_page or it_was_last_item: # approximate next page
        
        top_n_x_min_approx = counter.most_common(N_MOST_COMMON)
        df = pd.DataFrame(top_n_x_min_approx, columns = ["x_min_approx","freq"])
        df = df[df["freq"] > df["freq"].sum()*(1/(N_MOST_COMMON+1))].sort_values(by="x_min_approx")
        x_min_approx = (df["x_min_approx"] - LEFT_SECURITY_SHIFT).values
        x_min_approx = x_min_approx*(x_min_approx>0)
        left_x_min_suport = np.hstack([x_min_approx, 
                                      [10000]])
        def x_grouper(x_min):
            delta = left_x_min_suport-x_min
            x_group = left_x_min_suport[np.argmin(delta<0)*1-1]
            return x_group

        # iter on x_group and add items
        page_nb = idx_first_page + page_id
        column_text[page_nb] = {}

        for item in item_holder:
            (page_id, x_min, y_min, x_max, y_max, text) = item
            page_nb = idx_first_page + page_id
            x_group = x_grouper(x_min)
            if x_group in column_text[page_nb].keys():
                column_text[page_nb][x_group].append((y_min, y_max, text))
            else:
                column_text[page_nb][x_group] = [(y_min, y_max, text)]

        if it_was_last_item:
            break
        else:
            # restart from zero for next page
            counter = Counter()
            item_holder = []
            # update current_page_id
            current_page_id = page_id


Wall time: 0 ns


In [46]:
print(left_x_min_suport)

for i in range(1,1000, 50):
    print(i, x_grouper(i))

[   30.   290. 10000.]
1 10000.0
51 30.0
101 30.0
151 30.0
201 30.0
251 30.0
301 290.0
351 290.0
401 290.0
451 290.0
501 290.0
551 290.0
601 290.0
651 290.0
701 290.0
751 290.0
801 290.0
851 290.0
901 290.0
951 290.0


In [47]:
# CREATE THE PARAGRAPHS IN EACH COLUMN
# define minimal conditions to define a change of paragraph:
# Being spaced by more than the size of each line (min if different to account for titles)
pararaphs_list = []
paragraph_index = 0
for page_nb, x_groups_dict in column_text.items():
    for x_group_name, x_groups_data in x_groups_dict.items():
        x_groups_data = sorted(x_groups_data, key=lambda x: x[0],
                               reverse=True)  # sort vertically, higher y = before
        x_groups_data_paragraphs = []

        p = {"y_min": x_groups_data[0][0],
             "y_max": x_groups_data[0][1],
             "paragraph": x_groups_data[0][2]}

        previous_height = p["y_max"] - p["y_min"]
        previous_y_min = p["y_min"]
        for y_min, y_max, paragraph in x_groups_data[1:]:
            current_height = y_max - y_min
            current_y_min = y_min
            max_height = max(previous_height, current_height)

            relative_var_in_height = (current_height - previous_height) / float(
                current_height)  # Was min before ???
            relative_var_in_y_min = abs(current_y_min - previous_y_min) / float(current_height)

            positive_change_in_font_size = (relative_var_in_height > 0.05)
            change_in_font_size = abs(relative_var_in_height) > 0.05
            different_row = (relative_var_in_y_min > 0.7)
            large_gap = (relative_var_in_y_min > 1.2)
            artefact_to_ignore = (len(paragraph) <= 2)  # single "P" broke row parsing in auchan dpef
            if not artefact_to_ignore:
                if (positive_change_in_font_size and different_row) or large_gap:  # always break
                    # break paragraph, start new one
                    # print("break",relative_var_in_height, relative_var_in_y_min, paragraph)
                    p = clean_paragraph(p)
                    x_groups_data_paragraphs.append(p)
                    p = {"y_min": y_min,
                         "y_max": y_max,
                         "paragraph": paragraph}
                else:
                    # if change_in_font_size:  # to separate titles
                    #     paragraph = paragraph + ".\n"
                    # paragraph continues
                    p["y_min"] = y_min
                    p["paragraph"] = p["paragraph"] + " " + paragraph

                previous_height = current_height
                previous_y_min = current_y_min
        # add the last paragraph of column
        p = clean_paragraph(p)
        x_groups_data_paragraphs.append(p)
        # structure the output
        for p in x_groups_data_paragraphs:
            pararaphs_list.append({"paragraph_id": paragraph_index,
                                   "page_nb": page_nb,
                                   "x_group": x_group_name,
                                   "y_min_paragraph": round(p["y_min"]),
                                   "y_max_paragraph": round(p["y_max"]),
                                   "paragraph": p["paragraph"]})
            paragraph_index += 1
df_par = pd.DataFrame(data=pararaphs_list,
                      columns=["paragraph_id",
                               "page_nb",
                               "paragraph",
                               "x_group",
                               "y_min_paragraph",
                               "y_max_paragraph"])

In [49]:
df_par.to_csv("debug_edf.csv", encoding="utf-8-sig", sep=";")

In [44]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', None)
df_par.sort_values(by="x_group")

,paragraph_id,page_nb,paragraph,x_group,y_min_paragraph,y_max_paragraph
50,50,1,(2) Enedis est une filiale gérée en toute indépendance. (2) Enedis est une filiale gérée en toute indépendance.,20.0,35,43
191,191,3,"Les six Objectifs de Responsabilité d’Entreprise (ORE) traduisent des engagements à long terme (2030), dont l’exigence de mise en œuvre est déclinée dans les lettres de cadrage précisant la contribution attendue de chacune des entités et filiales du Groupe à la réussite de l’ambition commune. Le dispositif de pilotage de ces engagements est intégré à la boucle stratégique du Groupe. Les revues de performance annuelles permettent de suivre et contrôler leur réalisation effective par les entités et filiales.",20.0,589,656
192,192,3,"De même, les projets et investissements soumis à l’approbation du Comité des engagements du Comité exécutif du Groupe (2) (CECEG) font l’objet d’un avis spécifique de la Direction du Développement Durable élaboré à partir d’une grille de criblage traduisant en termes opérationnels les enjeux des six Objectifs de Responsabilité d’Entreprise. Lorsque nécessaire, la Direction du Développement Durable procède à ou organise des due diligences spécifiques à ces enjeux.",20.0,528,585
193,193,3,3.1.3 LA GOUVERNANCE DU DÉVELOPPEMENT DURABLE,20.0,486,512
51,51,1,"(3) Outre l’enquête interne « MyEDF »», cf. section 3.4 « Autres thématiques du domaine ressources humaines ». (3) Outre l’enquête interne « MyEDF »», cf. section 3.4 « Autres thématiques du domaine ressources humaines ».",20.0,26,33
...,...,...,...,...,...,...
80,80,1,EDF I Document de référence 2018 151,320.0,21,31
75,75,1,DIRECTIONS ET ENTITÉS DU GROUPE,320.0,438,459
253,253,4,3. INFORMATIONS ENVIRONNEMENTALES ET SOCIÉTALES − RESSOURCES HUMAINES,10000.0,790,827
131,131,2,3. INFORMATIONS ENVIRONNEMENTALES ET SOCIÉTALES − RESSOURCES HUMAINES 3. INFORMATIONS ENVIRONNEMENTALES ET SOCIÉTALES − RESSOURCES HUMAINES,10000.0,790,827


In [95]:
( (pd.DataFrame(x_min_list, columns = ["x_min"])//5)*5 ).groupby("x_min").size().sort_values(ascending=False)

x_min
55.0     28
310.0    24
100.0     4
65.0      2
165.0     1
80.0      1
70.0      1
40.0      1
25.0      1
dtype: int64

In [50]:
column_text

OrderedDict([(0,
              {0: [(790.4401,
                 826.5601,
                 '3. INFORMATIONS ENVIRONNEMENTALES ET SOCIÉTALES − RESSOURCES HUMAINES'),
                (785.4105,
                 795.483,
                 'Le nouveau contexte du reporting extra-financier'),
                (732.894,
                 749.75,
                 'LE NOUVEAU CONTEXTE DU REPORTING EXTRA-FINANCIER'),
                (705.534,
                 714.838,
                 'L’ordonnance 2017-1180 du 19 juillet 2017 et le décret 2017-1265 du 9 août 2017'),
                (695.934,
                 705.2379999999999,
                 'ont transposé la directive européenne 2014/95/UE modifiant l’article 225 du Code'),
                (686.334,
                 695.6379999999999,
                 'de Commerce qui impose aux entreprises relevant de son champ d’application de'),
                (676.7339999999999,
                 686.0379999999999,
                 'reporter sur les inform

In [47]:
df_par = get_paragraphs_from_raw_content(device, idx_first_page)

In [91]:
df_par[df_par.page_nb==0].shape

(15, 6)